<a href="https://colab.research.google.com/github/x1001000/Colab-Notebooks/blob/main/Hello_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download [Llama 2](https://ai.meta.com/llama/) paper (published on 2023/7/18) from arXiv to Drive as an example

In [2]:
!mkdir /content/drive/MyDrive/LangChain_demo
%cd /content/drive/MyDrive/LangChain_demo
!curl https://arxiv.org/pdf/2307.09288.pdf -O

/content/drive/MyDrive/LangChain_demo
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.0M  100 13.0M    0     0  1248k      0  0:00:10  0:00:10 --:--:-- 1498k


# Install libraries

In [3]:
!pip install langchain --quiet
!pip install python-dotenv --quiet
!pip install openai --quiet
!pip install chromadb --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.0 MB/s

# Import libraries


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
# from langchain.document_loaders import GoogleDriveLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

# Load your OpenAI API key

In [5]:
!echo OPENAI_API_KEY='sk-TrakhGZqZyBT4g0eTrYPZBlbkFJsVnRMfFT3d69stNLckw4C'>.env
load_dotenv()

True

# Load your docs in LangChain_demo folder

In [6]:
loader = DirectoryLoader('/content/drive/MyDrive/LangChain_demo')
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Use the code from [Joe](https://www.meetup.com/taipei_langchain/events/294388359/?utm_medium=referral&utm_campaign=share-btn_savedevents_share_modal&utm_source=link)

In [7]:
# The beauty of Langchain.. all of the heavy lifting is done!
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 4000, chunk_overlap = 0, separators = [" ", ",", "\n"])
texts = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

# your assistant's personality
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k") # the "brain"
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever) # the behavior

# Chat with your PDFs

In [8]:
qa.run('請告訴我 LLAMA 2 論文的前三位作者？')

'LLAMA 2 論文的前三位作者是 Hugo Touvron、Louis Martin 和 Kevin Stone。'